In [83]:
from sentence_transformers import SentenceTransformer, util
from nltk.corpus import stopwords
from tqdm import tqdm
import torch
#import spacy
import dill
import re
import pandas as pd

In [ ]:
nlp = spacy.load("ru_core_news_sm")

### Очистка данных

In [19]:
def find_experience(experience):
    if experience=='Не указано':
        return -1
    else:
        result_exp = re.findall(pattern, experience)[0]
        mnth = int(result_exp[0])*12 + int(result_exp[1])
        return mnth
    
def experience_to_group(months):
    if months < 12:
        return 'Нет опыта'
    elif 12 <= months < 36:
        return 'От 1 года до 3 лет'
    elif 36 <= months < 72:
        return 'От 3 до 6 лет'
    else:
        return 'Более 6 лет'

def exp_group(value):
    if value=='Нет опыта':
        return 1
    elif value=='От 1 года до 3 лет':
        return 2
    elif value=='От 3 до 6 лет':
        return 3
    else:
        return 4
    
def extract_skills(text):
    doc = nlp(text)
    skills = [token.text for token in doc if token.pos_ in {'NOUN', 'ADJ'} and token.text.lower() not in stop_words]
    return ' '.join(skills)

In [30]:
columns = ['title', 'location', 'salary', 'experience_group', 'job_type', 'description']

ban_words = ['руб', 'месяц', '/', 'июнь', 'июль', 'август', 'сентябрь', 'октябрь', 'декабрь', 'январь', 'февраль', 'март', 'апрель','опыт', 'работы', 'месяца', 'месяцев', 'год', 'года', 'лет','настоящее','время']

stop_words = set(stopwords.words('russian') + ban_words)

#### Резюме

In [60]:
cv = pd.read_csv('data/hh_cv.csv', on_bad_lines='skip', encoding='utf-8', sep=';')

cv['job_type'] = cv['Занятость'] + ', ' + cv['График']

cv = cv.drop(['Пол, возраст', 'Обновление резюме', 'Авто', 'Последнее/нынешнее место работы', 'Последняя/нынешняя должность', 'Образование и ВУЗ', 'Занятость', 'График'], axis=1)

cv = cv.rename(columns={
    'Ищет работу на должность:':'title',
    'ЗП':'salary',
    'Город, переезд, командировки':'location',
    'График':'job_type',
    'Опыт работы':'experience',
    'Последнее/нынешнее место работы':'last_job'
})

cv.salary = cv.salary.apply(lambda value: value.split(' ')[0])

cv.location = cv.location.apply(lambda value: value.split(' ')[0])

cv.experience = cv.experience.fillna('0 0')
cv['experience_month'] = cv.apply(lambda row: find_experience(row.experience), axis=1)
cv['experience_group'] = cv.experience_month.apply(experience_to_group)

cv = cv.rename(columns={
    'experience_clean_list':'description'
})

cv.salary = cv.salary.astype('float64')

cv.job_type = cv.job_type.apply(lambda value: value.capitalize())

cv.description = cv.description.fillna('')
cv.description = cv.description.apply(lambda value: ''.join([i for i in value if not i.isdigit()]))
cv.description = cv.description.apply(lambda value: ' '.join(value.split()))
cv.description = cv.description.apply(lambda value: ' '.join([i for i in value.split(' ') if i not in ban_words]))

cv['exp_group_num'] = cv.experience_group.apply(exp_group)

cv['description'] = [extract_skills(i) for i in tqdm(cv.experience.values)]

cv = cv[cols]

In [67]:
cv.head()

,title,location,salary,experience_group,job_type,description
0,Системный администратор,Советск,29000.0,Более 6 лет,"Частичная занятость, проектная работа, полная ...",СОШ г Немана Системный администратор установка...
1,Технический писатель,Королев,40000.0,Более 6 лет,"Частичная занятость, проектная работа, полная ...",Временный трудовой коллектив Информационные те...
2,Оператор,Тверь,20000.0,Более 6 лет,"Полная занятость, полный день",операционист денежной наличностью обслуживание...
3,Веб-разработчик (HTML / CSS / JS / PHP / базы ...,Саратов,100000.0,Более 6 лет,"Частичная занятость, проектная работа, полная ...",Информационные технологии системная интеграция...
4,Региональный менеджер по продажам,Москва,140000.0,От 3 до 6 лет,"Полная занятость, полный день",Региональный менеджер продажам Информационные ...


In [77]:
cv.to_csv('cv_clean.csv')

#### Вакансии

In [84]:
vacancy = pd.read_csv('data/hh_vacancy.csv', on_bad_lines='skip', encoding='utf-8', sep=';')

vacancy = vacancy.dropna()

vacancy = vacancy.drop(['company','date_of_post','type','№','id', 'description'], axis=1)

vacancy = vacancy.rename(columns={
    'experience': 'experience_group',
    'key_skills': 'description'
})

def extract_average_salary(salary_range):
    # Поиск числовых значений в строке
    numbers = [int(match.group()) for match in re.finditer(r'\b\d+\b', salary_range)]
    
    # Если числа найдены, возвращаем среднее значение, иначе возвращаем None
    if numbers:
        return sum(numbers) / len(numbers)
    else:
        return None
    
vacancy.salary = vacancy.salary.apply(extract_average_salary)

vacancy.location = vacancy.location.apply(lambda value: value.replace('г.', '').split(',')[0])

vacancy = vacancy.dropna()

vacancy['exp_group_num'] = vacancy.experience_group.apply(exp_group)

vacancy.description = vacancy.description.str.replace(',',', ')
vacancy.job_type = vacancy.job_type.str.replace(',',', ')

vacancy = vacancy[cols]

vacancy = vacancy.reset_index().drop('index', axis=1)

In [ ]:
vacancy.head()

In [ ]:
vacancy.to_csv('data/vacancy_clean.csv')

#### Получение эмбеддингов

In [16]:
cv = pd.read_csv('data/cv_clean.csv', index_col=0)

In [21]:
vacancy = pd.read_csv('data/vacancy_clean.csv', index_col=0)

In [22]:
model_name = 'intfloat/multilingual-e5-large'

model = SentenceTransformer(model_name)

In [23]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Using device: {device}')

Using device: cuda


In [100]:
%%time

cv_embedding_e5 = model.encode(cv.apply(lambda row: '. '.join(row.astype(str)), axis=1), convert_to_tensor=True, device='cuda')

In [97]:
with open('emb/cv_embedding_e5.dill', 'wb') as f:
    dill.dump(cv_embedding_e5, f)

In [99]:
%%time

vc_embedding_e5 = model.encode(vacancy.apply(lambda row: '. '.join(row.astype(str)), axis=1), convert_to_tensor=True, device='cuda')

In [98]:
with open('emb/vacancy_embedding_e5.dill', 'wb') as f:
    dill.dump(vc_embedding_e5, f)

In [27]:
%%time

cv_embedding_e5_small = model.encode(cv.drop('description', axis=1).apply(lambda row: '. '.join(row.astype(str)), axis=1), convert_to_tensor=True, device='cuda')

CPU times: total: 1min 28s
Wall time: 1min 18s


In [29]:
cv_embedding_e5_small[0].shape

torch.Size([1024])

In [28]:
with open('emb/cv_embedding_e5_small.dill', 'wb') as f:
    dill.dump(cv_embedding_e5_small, f)

#### Пример использования

In [ ]:
ban_words = ['опыт', 'работы', 'месяца', 'месяцев', 'год', 'года', 'лет','настоящее','время', 'руб', 'месяц', 'июнь', 'июль', 'август', 'сентябрь', 'октябрь', 'декабрь', 'январь', 'февраль', 'март', 'апрель']

stop_words = set(stopwords.words('russian') + ban_words)

def extract_skills(text):
    doc = nlp(text)
    skills = [token.text for token in doc if token.pos_ in {'NOUN', 'ADJ'} and token.text.lower() not in stop_words]
    return skills

def sem_search_results_cv(model, base_query, corpus, corpus_embedding, top_k):
    query_list = extract_skills(base_query.split('\n')[-1]) + base_query.split('\n')[:-1]
    query = ' '.join(query_list)
    
    question_embedding = model.encode(query, convert_to_tensor=True)
    hits = util.semantic_search(question_embedding, corpus_embedding, top_k=top_k)
    hits = hits[0] 

    print(base_query)
    print()
    cross_inp_res = [[corpus[hit['corpus_id']], hit['score']] for hit in hits]
    cross_inp_emb = [question_embedding[0]] + [corpus_embedding[hit['corpus_id']] for hit in hits]
    return cross_inp_res

def sem_search_results_vc(model, base_query, corpus, corpus_embedding, top_k):
    question_embedding = model.encode(base_query, convert_to_tensor=True)
    hits = util.semantic_search(question_embedding, corpus_embedding, top_k=top_k)
    hits = hits[0] 

    print(base_query)
    print()
    cross_inp_res = [[corpus[hit['corpus_id']], hit['score']] for hit in hits]
    cross_inp_emb = [question_embedding[0]] + [corpus_embedding[hit['corpus_id']] for hit in hits]
    return cross_inp_res

In [117]:
cv_list = cv.apply(lambda row: '. '.join(row.astype(str)), axis=1).to_list()
vacancy_list = vacancy.apply(lambda row: '. '.join(row.astype(str)), axis=1).to_list()

In [159]:
cv_exmaple = \
"""Системный администратор.
Советск. 29000.0. Более 6 лет.
Полная занятость, гибкий график, полный день.
Консультирование клиентов компании. Поддержание в рабочем состоянии компьютерной техники и программного обеспечения информационного отдела Разработка и поддержка локального программного обеспечения отдела. Сборка компьютеров для новых сотрудников."""

In [183]:
sem_search_results(model, cv_exmaple, vacancy_list, vc_embedding_e5, 3)

Системный администратор.
Советск. 29000.0. Более 6 лет.
Полная занятость, гибкий график, полный день.
Консультирование клиентов компании. Поддержание в рабочем состоянии компьютерной техники и программного обеспечения информационного отдела Разработка и поддержка локального программного обеспечения отдела. Сборка компьютеров для новых сотрудников.



[['Системный администратор. Курск. 50000.0. От 1 года до 3 лет. Полная занятость, полный день. Настройка ПК, Сборка ПК, Настройка сетевых подключений, Настройка ПО, Организация рабочих мест',
  0.9202420115470886],
 ['Помощник системного администратора. Новосибирск. 60000.0. От 1 года до 3 лет. Полная занятость, полный день. Сборка ПК, Настройка ПК, Администрирование сетевого оборудования, Монтаж ЛВС, Телефония, Умение работать в коллективе, Системы видеонаблюдения',
  0.9201076030731201],
 ['Системный администратор. Москва. 150000.0. От 3 до 6 лет. Полная занятость, полный день. Настройка ПК, Настройка ПО, Администрирование сайтов, Техническая поддержка, Обеспечение антивирусной защиты, Администрирование сетевого оборудования, Информационные технологии, Администрирование серверов, Информационная безопасность',
  0.9199874997138977]]

In [198]:
vc_example = """
Управляющий рестораном. 
Москва. 100000.0. 
От 3 до 6 лет. 
Полная занятость, полный день. 
Умение работать в условиях многозадачности.
"""

In [201]:
cv

,title,location,salary,experience_group,job_type,description
0,Системный администратор,Советск,29000.0,Более 6 лет,"Частичная занятость, проектная работа, полная ...",СОШ г Немана Системный администратор установка...
1,Технический писатель,Королев,40000.0,Более 6 лет,"Частичная занятость, проектная работа, полная ...",Временный трудовой коллектив Информационные те...
2,Оператор,Тверь,20000.0,Более 6 лет,"Полная занятость, полный день",операционист денежной наличностью обслуживание...
3,Веб-разработчик (HTML / CSS / JS / PHP / базы ...,Саратов,100000.0,Более 6 лет,"Частичная занятость, проектная работа, полная ...",Информационные технологии системная интеграция...
4,Региональный менеджер по продажам,Москва,140000.0,От 3 до 6 лет,"Полная занятость, полный день",Региональный менеджер продажам Информационные ...
...,...,...,...,...,...,...
44739,"Финансист, аналитик, экономист, бухгалтер, мен...",Тверь,50000.0,Более 6 лет,"Полная занятость, полный день, удаленная работа",аналитик экономист бухгалтер менеджер Информац...
44740,"Системный администратор, IT-специалист",Липецк,39000.0,Более 6 лет,"Проектная работа, частичная занятость, полная ...",Системный администратор Информационные техноло...
44741,"Аналитик данных, Математик",Челябинск,40000.0,Более 6 лет,"Полная занятость, полный день, удаленная работа",ОАО ЧМК Исследовательско - Технологический Нач...
44742,Контент-менеджер,Тамбов,20000.0,От 3 до 6 лет,"Частичная занятость, полная занятость, удаленн...",Контент - менеджер Информационные технологии и...


In [200]:
cv_base = pd.read_csv('data/hh_cv.csv', on_bad_lines='skip', encoding='utf-8', sep=';')

In [207]:
cv['description_base'] = cv_base['Опыт работы']

In [199]:
sem_search_results_vc(model, vc_example, cv_list, cv_embedding_e5, 3)


Управляющий рестораном. 
Москва. 100000.0. 
От 3 до 6 лет. 
Полная занятость, полный день. 
Умение работать в условиях многозадачности.




[['Менеджер. Москва. 100000.0. От 3 до 6 лет. Полная занятость, полный день. Информационные технологии интернет телеком полная занятость полный день Ведущий',
  0.9219639897346497],
 ['Администратор. Москва. 10000.0. Более 6 лет. Полная занятость, полный день. кафе работа клиентами разрешение проблем клиентами работа кассой',
  0.9217270016670227],
 ['управляющий. Москва. 80000.0. Более 6 лет. Полная занятость, полный день. силы персоналом армии навыки пайки замены контроллера питания телефоне электрике Дипломатичный подход решению конфликтных ситуаций Готов задачи управления подход клиенту компании фирме конкурентам нестандартные задачи выход',
  0.9205193519592285]]

In [213]:
vacancy_list = vacancy.apply(lambda row: '\n'.join(row.astype(str)), axis=1).to_list()

In [214]:
cv_list = cv.drop('description', axis=1).apply(lambda row: '\n'.join(row.astype(str)), axis=1).to_list()

In [221]:
cv_list_small = cv.drop(['description','description_base'], axis=1).apply(lambda row: '\n'.join(row.astype(str)), axis=1).to_list()

In [215]:
with open('emb/vacancy_list.dill', 'wb') as f:
    dill.dump(vacancy_list, f)

In [217]:
with open('emb/cv_list.dill', 'wb') as f:
    dill.dump(cv_list, f)

In [222]:
with open('emb/cv_list_small.dill', 'wb') as f:
    dill.dump(cv_list_small, f)

### qdrant

In [30]:
from qdrant_client import models
from qdrant_client import QdrantClient
from qdrant_client.http.models import Distance, VectorParams

import dill

In [31]:
with open("emb/vacancy_embedding_e5.dill", "rb") as f:
    vacancy_embedding_e5 = dill.load(f)
    
with open("emb/cv_embedding_e5.dill", "rb") as f:
    cv_embedding_e5 = dill.load(f)

In [32]:
with open("emb/vacancy_list.dill", "rb") as f:
    vacancy_list = dill.load(f)
    
with open("emb/cv_list.dill", "rb") as f:
    cv_list = dill.load(f)

In [33]:
qdrant_client = QdrantClient(
    url="https://c457e10a-65e6-4023-95fb-5e84bcf5fc5f.us-east4-0.gcp.cloud.qdrant.io:6333", 
    api_key="znIMw4rgEny3fri-SeMNk7QSuCcOQ3VLrX4_z6cEEklxgZPGhxyZgw",
)

In [75]:
qdrant_client.get_collections()

CollectionsResponse(collections=[CollectionDescription(name='cv_vectors'), CollectionDescription(name='vc_vectors')])

In [46]:
#qdrant_client.delete_collection(collection_name="cv_vectors")

True

In [6]:
cv_vector_lists = [vector.tolist() for vector in cv_embedding_e5]

cv_names = [i.split('\n') for i in cv_list]

In [52]:
cv_records=[
        models.Record(
            id=idx, vector=doc, payload={'title':cv_names[idx]})
        for idx, doc in enumerate(cv_vector_lists)
    ]

qdrant_client.recreate_collection(
    collection_name="cv_vectors",
    vectors_config=VectorParams(
        size=1024, 
        distance=models.Distance.COSINE,
    ),
)

qdrant_client.upload_records(
    collection_name="cv_vectors",
    records=cv_records,
)

In [38]:
vc_vector_lists = [vector.tolist() for vector in vacancy_embedding_e5]

vc_names = [i.split('\n')[0] for i in vacancy_list]

In [39]:
#Обрезаем вакансии
N = 40000

vc_vector_lists = vc_vector_lists[:N]

In [40]:
vc_records=[
        models.Record(
            id=idx, vector=doc, payload={'title':vc_names[idx]})
        for idx, doc in enumerate(vc_vector_lists)
    ]

qdrant_client.recreate_collection(
    collection_name="vc_vectors",
    vectors_config=VectorParams(
        size=1024, 
        distance=models.Distance.COSINE,
    ),
)

qdrant_client.upload_records(
    collection_name="vc_vectors",
    records=vc_records,
)

### Поиск в qdrant

In [55]:
from sentence_transformers import SentenceTransformer, util

In [70]:
with open("emb/cv_list.dill", "rb") as f:
    cv_list = dill.load(f)

In [81]:
with open("emb/vacancy_list.dill", "rb") as f:
    vacancy_list = dill.load(f)

In [56]:
qdrant_client = QdrantClient(
    url="https://c457e10a-65e6-4023-95fb-5e84bcf5fc5f.us-east4-0.gcp.cloud.qdrant.io:6333", 
    api_key="znIMw4rgEny3fri-SeMNk7QSuCcOQ3VLrX4_z6cEEklxgZPGhxyZgw",
)

In [57]:
model_name = 'intfloat/multilingual-e5-large'

model = SentenceTransformer(model_name)

#### Резюме по вакансии

In [78]:
query = """
Управляющий рестораном. 
Москва. 100000.0. 
От 3 до 6 лет. 
Полная занятость, полный день. 
Умение работать в условиях многозадачности. Ответственность. Целеустремленность.
"""

query_embedding = model.encode(query, convert_to_tensor=True).tolist()

hits = qdrant_client.search(
    collection_name="cv_vectors",
    query_vector=query_embedding,
    limit=5,
)

for hit in hits:
    print(hit.id, hit.payload, "score:", hit.score)
    print(cv_list[hit.id])
    print()

27030 {'title': 'управляющий'} score: 0.9184449
управляющий
Москва
80000.0
Более 6 лет
Полная занятость, полный день
Опыт работы 19 лет 5 месяцев  Январь 2000 — по настоящее время 19 лет 5 месяцев вооруженные силы Управляющий персоналом Опыт 6 лет работы в армии. Есть навыки пайки вплоть до замены контроллера питания на телефоне. Хорошо понимаю в Электронике и электрике. Дипломатичный подход к решению конфликтных ситуаций. Готов решать задачи управления Коллективом и искать подход к клиенту, компании, фирме, конкурентам. Решать нестандартные задачи. Доказать, что выход в любой из них всегда существует.

4997 {'title': 'Менеджер'} score: 0.9159361
Менеджер
Москва
100000.0
От 3 до 6 лет
Полная занятость, полный день
Опыт работы 4 года 7 месяцев  Менеджер 100 000 руб. Информационные технологии, интернет, телеком Продажи Занятость: полная занятость График работы: полный день Опыт работы 4 года 7 месяцев Май 2018 — Август  2018 4 месяца ООО «НПЦ Трезор» Россия Ведущий и

4585 {'title': 'Адм

#### Вакансии по резюме

In [ ]:
nlp = spacy.load("ru_core_news_sm")

ban_words = ['руб', 'месяц', '/', 'июнь', 'июль', 'август', 'сентябрь', 'октябрь', 'декабрь', 'январь', 'февраль', 'март', 'апрель','опыт', 'работы', 'месяца', 'месяцев', 'год', 'года', 'лет','настоящее','время']

stop_words = set(stopwords.words('russian') + ban_words)

def extract_skills(text):
    """Функция очистки"""
    doc = nlp(text)
    skills = [token.text for token in doc if token.pos_ in {'NOUN', 'ADJ'} and token.text.lower() not in stop_words]
    return ' '.join(skills)

In [82]:
query = """
Системный администратор.
Советск. 29000.0. Более 6 лет.
Полная занятость, гибкий график, полный день.
Консультирование клиентов компании. Поддержание в рабочем состоянии компьютерной техники и программного обеспечения информационного отдела Разработка и поддержка локального программного обеспечения отдела. Сборка компьютеров для новых сотрудников.
"""

#Отказывается работать, так как pydantic начал конфликтовать с qdrant. Из-за этого не могу импортировать spacy для очистки.
#query = extract_skills(query)

query_embedding = model.encode(query, convert_to_tensor=True).tolist()

hits = qdrant_client.search(
    collection_name="vc_vectors",
    query_vector=query_embedding,
    limit=3,
)

for hit in hits:
    print(hit.id, hit.payload, "score:", hit.score)
    print(vacancy_list[hit.id])
    print()

22410 {'title': 'Системный администратор'} score: 0.92629385
Системный администратор
Санкт-Петербург
60000.0
От 3 до 6 лет
Полная занятость, полный день
Администрирование сетевого оборудования, Техническое обслуживание, Настройка ПО, Настройка ПК, Закупка оргтехники и оборудования, Сборка ПК, Ремонт ПК, Техническая поддержка, Администрирование серверов, Обеспечение антивирусной защиты, Офисная техника, Антивирусная защита сети, Организация рабочих мест

31926 {'title': 'Системный администратор'} score: 0.92471063
Системный администратор
Ноябрьск
60000.0
От 3 до 6 лет
Полная занятость, полный день
Работа в команде, оформление документации, Работа в условиях многозадачности, Работа с компьютером, Коммуникабельность,  ответственность

25306 {'title': 'Системный администратор'} score: 0.92409396
Системный администратор
Новосибирск
40000.0
От 1 года до 3 лет
Полная занятость, полный день
Организация рабочих мест, Сборка ПК, Образованность, Ответственный подход к работе, Настройка сетевых по